In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from lazypredict.Supervised import LazyClassifier

In [2]:
# load dataset

train = pd.read_csv("./rs-data-science-wise-202324/train.csv")
test = pd.read_csv("./rs-data-science-wise-202324/test.csv")

In [3]:
train.head()

,UniqueID,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
0,0,3291.00,358.00,18.00,30.00,10.00,2161.00,189.00,204.00,150.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1
1,1,2563.00,334.00,18.00,60.00,5.00,1084.00,177.00,214.00,174.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2
2,2,3340.00,187.00,24.00,90.00,21.00,1041.00,215.00,NaN,154.00,...,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,7
3,3,2922.00,103.00,4.00,67.00,3.00,5057.00,227.00,234.00,141.00,...,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0.00,0.00,1
4,4,2728.00,15.00,15.00,446.00,86.00,NaN,205.00,207.00,136.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2


In [4]:
train.columns

Index(['UniqueID', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3',
       'Wilderness_Area_4', 'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3',
       'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7',
       'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11',
       'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15',
       'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19',
       'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23',
       'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27',
       'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31',
       'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35',
       'Soil_Type_36', 'Soil

In [5]:
# extract binary features
binary_features = [col for col in train.columns if train[col].nunique() == 2]

# extract categorical features (i.e., integer features with <= 10 unique values)
categorical_features = [
    col for col in train.columns if train[col].dtype == "int64" and train[col].nunique() <= 10
]

# extract numerical features
numerical_features = [
    col for col in train.columns if train[col].dtype in ["int64", "float64"] and col not in binary_features
]


In [6]:
numerical_features

['UniqueID',
 'Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Soil_Type_15',
 'Cover_Type']

In [7]:
categorical_features

['Cover_Type']

In [8]:
binary_features

['Wilderness_Area_1',
 'Wilderness_Area_2',
 'Wilderness_Area_3',
 'Wilderness_Area_4',
 'Soil_Type_1',
 'Soil_Type_2',
 'Soil_Type_3',
 'Soil_Type_4',
 'Soil_Type_5',
 'Soil_Type_6',
 'Soil_Type_7',
 'Soil_Type_8',
 'Soil_Type_9',
 'Soil_Type_10',
 'Soil_Type_11',
 'Soil_Type_12',
 'Soil_Type_13',
 'Soil_Type_14',
 'Soil_Type_16',
 'Soil_Type_17',
 'Soil_Type_18',
 'Soil_Type_19',
 'Soil_Type_20',
 'Soil_Type_21',
 'Soil_Type_22',
 'Soil_Type_23',
 'Soil_Type_24',
 'Soil_Type_25',
 'Soil_Type_26',
 'Soil_Type_27',
 'Soil_Type_28',
 'Soil_Type_29',
 'Soil_Type_30',
 'Soil_Type_31',
 'Soil_Type_32',
 'Soil_Type_33',
 'Soil_Type_34',
 'Soil_Type_35',
 'Soil_Type_36',
 'Soil_Type_37',
 'Soil_Type_38',
 'Soil_Type_39',
 'Soil_Type_40']

In [9]:
train['Soil_Type_15'].value_counts()

Soil_Type_15
0.00    9634
Name: count, dtype: int64

In [10]:
# add soil type 15 to binary features
binary_features.append('Soil_Type_15')

In [11]:
soil_type_features = [col for col in train.columns if col.startswith('Soil_Type')]
wilderness_area_features = [col for col in train.columns if col.startswith('Wilderness_Area')]


In [12]:
# remove cover type from categorical features
categorical_features.remove('Cover_Type')

In [13]:
# drop nan rows
train = train.dropna()

In [15]:
# take the binary features of the soil type and make them one categorical feature
train['Soil_Type'] = train[soil_type_features].idxmax(axis=1)

# take the binary features of the wilderness area and make them one categorical feature
train['Wilderness_Area'] = train[wilderness_area_features].idxmax(axis=1)

# drop the binary features of the soil type
train = train.drop(soil_type_features, axis=1)

# drop the binary features of the wilderness area
train = train.drop(wilderness_area_features, axis=1)

In [16]:
train.head()

,UniqueID,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type,Soil_Type,Wilderness_Area
0,0,3291.00,358.00,18.00,30.00,10.00,2161.00,189.00,204.00,150.00,1405.00,1,Soil_Type_38,Wilderness_Area_3
1,1,2563.00,334.00,18.00,60.00,5.00,1084.00,177.00,214.00,174.00,577.00,2,Soil_Type_29,Wilderness_Area_1
7,7,3077.00,64.00,13.00,60.00,12.00,5079.00,232.00,212.00,110.00,757.00,1,Soil_Type_30,Wilderness_Area_1
10,10,2407.00,300.00,22.00,182.00,63.00,1024.00,155.00,225.00,209.00,1518.00,6,Soil_Type_10,Wilderness_Area_4
11,11,2801.00,237.00,18.00,30.00,5.00,2507.00,186.00,253.00,201.00,470.00,2,Soil_Type_13,Wilderness_Area_3


In [17]:
X = train.drop(['Cover_Type', 'UniqueID'], axis=1)
y = train['Cover_Type']

In [18]:
# split data into train and validation sets
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# convert categorical features to string and binary features to bool
#for col in categorical_features:
#    X_train[col] = X_train[col].astype(str)
#    X_valid[col] = X_valid[col].astype(str)

#for col in binary_features:
#    X_train[col] = X_train[col].astype(bool)
#    X_valid[col] = X_valid[col].astype(bool)

In [23]:
# train model using lazy classifier and dont normalize data binary_features and categorical_features
# use F1 score as metric to rank models
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_valid, y_train, y_valid)

models.sort_values('F1 Score', ascending=False)

 93%|████████████████████████████████████████   | 27/29 [00:03<00:00,  7.68it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000181 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1987
[LightGBM] [Info] Number of data points in the train set: 2430, number of used features: 15
[LightGBM] [Info] Start training from score -1.006675
[LightGBM] [Info] Start training from score -0.723225
[LightGBM] [Info] Start training from score -2.839819
[LightGBM] [Info] Start training from score -4.962433
[LightGBM] [Info] Start training from score -4.184729
[LightGBM] [Info] Start training from score -3.668512
[LightGBM] [Info] Start training from score -3.141686
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

100%|███████████████████████████████████████████| 29/29 [00:04<00:00,  7.03it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.79,0.56,None,0.78,0.30
RandomForestClassifier,0.77,0.54,None,0.77,0.54
LGBMClassifier,0.76,0.54,None,0.76,0.38
BaggingClassifier,0.73,0.50,None,0.73,0.19
LogisticRegression,0.73,0.42,None,0.71,0.09
LabelPropagation,0.70,0.51,None,0.70,0.30
LabelSpreading,0.70,0.51,None,0.70,0.42
KNeighborsClassifier,0.70,0.50,None,0.69,0.06
DecisionTreeClassifier,0.69,0.52,None,0.69,0.05
